In [2]:
#Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://chalmers-mit-openai.openai.azure.com/"
openai.api_version = "2023-05-15"

with(open('openai.key')) as f:
    openai.api_key = f.read().strip()

In [3]:
from PyPDF2 import PdfReader
pdf_file = "example2"
reader = PdfReader(pdf_file+".pdf")
number_of_pages = len(reader.pages)
#pages = [page.extract_text() for page in reader.pages]

In [13]:
print(pages[1])

Parallel Programming


In [29]:
EMBEDDING_MODEL = "text-embedding-ada-002"
BATCH_SIZE = 1

embeddings = []
for batch_start in range(0, len(pages), BATCH_SIZE):
    batch_end = batch_start + BATCH_SIZE
    batch = pages[batch_start:batch_end]
    print(f"Batch {batch_start} to {batch_end-1}")
    response = openai.Embedding.create(model=EMBEDDING_MODEL, input=batch, deployment_id=EMBEDDING_MODEL)
    for i, be in enumerate(response["data"]):
        assert i == be["index"]  # double check embeddings are in same order as input
    batch_embeddings = [e["embedding"] for e in response["data"]]
    embeddings.extend(batch_embeddings)

Batch 0 to 0
Batch 1 to 1
Batch 2 to 2
Batch 3 to 3
Batch 4 to 4
Batch 5 to 5
Batch 6 to 6
Batch 7 to 7
Batch 8 to 8
Batch 9 to 9
Batch 10 to 10
Batch 11 to 11
Batch 12 to 12
Batch 13 to 13
Batch 14 to 14
Batch 15 to 15
Batch 16 to 16
Batch 17 to 17
Batch 18 to 18
Batch 19 to 19
Batch 20 to 20
Batch 21 to 21
Batch 22 to 22
Batch 23 to 23
Batch 24 to 24
Batch 25 to 25
Batch 26 to 26
Batch 27 to 27
Batch 28 to 28
Batch 29 to 29
Batch 30 to 30
Batch 31 to 31
Batch 32 to 32
Batch 33 to 33
Batch 34 to 34
Batch 35 to 35
Batch 36 to 36
Batch 37 to 37
Batch 38 to 38
Batch 39 to 39
Batch 40 to 40
Batch 41 to 41
Batch 42 to 42
Batch 43 to 43
Batch 44 to 44
Batch 45 to 45
Batch 46 to 46
Batch 47 to 47
Batch 48 to 48
Batch 49 to 49
Batch 50 to 50
Batch 51 to 51
Batch 52 to 52
Batch 53 to 53
Batch 54 to 54
Batch 55 to 55
Batch 56 to 56
Batch 57 to 57
Batch 58 to 58
Batch 59 to 59
Batch 60 to 60
Batch 61 to 61
Batch 62 to 62
Batch 63 to 63
Batch 64 to 64
Batch 65 to 65
Batch 66 to 66
Batch 67 to 67


In [1]:
import pandas as pd
df = pd.DataFrame({"text": pages, "embedding": embeddings})

SAVE_PATH = f"data/{pdf_file}.csv"

df.to_csv(SAVE_PATH, index=False)

NameError: name 'pages' is not defined

In [4]:
import pandas as pd
import ast
from scipy import spatial
pdf_file = "example2"
SAVE_PATH = f"data/{pdf_file}.csv"
df = pd.read_csv(SAVE_PATH)
df['embedding'] = df['embedding'].apply(ast.literal_eval)

def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple[list[str], list[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = openai.Embedding.create(
        model=EMBEDDING_MODEL,
        input=query,
        deployment_id=EMBEDDING_MODEL
    )
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

In [6]:
EMBEDDING_MODEL = "text-embedding-ada-002"
strings, relatednesses = strings_ranked_by_relatedness("State space models in bsp to capture important", df, top_n=5, openai= openai)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.810


'4.5 Parallel Computational Models 209\nThe BSP model has been extended to the Multi-BSP model, which extends the\noriginal BSP model to capture important characteristics of modern architectures, in\nparticular multicore architectures [194]. In particular, the model is extended to ahierarchical model with an arbitrary number dof levels modeling multiple memory\nand cache levels. Moreover, at each level the memory size is incorporated as an addi-\ntional parameter. The entire model is based on a tree of depth dwith memory/caches\nat the internal nodes and processors at the leaves.\n4.5.3 LogP Model\nIn [40], several concerns about the BSP model are formulated. First, the length\nof the supersteps must be sufﬁciently large to accommodate arbitrary h-relations.\nThis has the effect that the granularity cannot be decreased below a certain value.Second, messages sent within a superstep can only be used in the next superstep,\neven if the interconnection network is fast enough to deliver mes

relatedness=0.803


'4.5 Parallel Computational Models 207\n4.5.2 BSP Model\nNone of the PRAM models proposed has really been able to capture the behavior\nof real parallel machines for a large class of application areas in a satisfactory way.One of the reasons is that there is a large variety of different architectures for parallel\nmachines and the architectures are steadily evolving. To avoid that the computa-\ntional model design constantly drags behind the development of parallel computerarchitecture, the BSP model ( bulk synchronously parallel ) has been proposed as a\nbridging model between hardware architecture and software development [193]. The\nidea is to provide a standard on which both hardware architects and software devel-opers can agree. Thus, software development can be decoupled from the details of a\nspeciﬁc architecture, and software does not have to be adapted when porting it to a\nnew parallel machine.\nThe BSP model is an abstraction of a parallel machine with a physically distrib-\

relatedness=0.795


'208 4 Performance Analysis of Parallel Programs\nbarrier synchronization\nbarrier synchronization\ntimesupersteplocal comp utations\nglobal comm unication\nvirtual processors\nFig. 4.8 In the BSP model, computations are performed in supersteps where each superstep consists\nof three phases: (1) simultaneous local computations of each processor, (2) communication oper-ations for data exchange between processors, and (3) a barrier synchronization to terminate thecommunication operations and to make the data sent visible to the receiving processors. The com-munication pattern shown for the communication phase represents an h-relation with h =3.\nThe parameter gis determined such that the execution of an h-relation with m\nwords per message takes l·m·gsteps. For a real parallel computer, the value of\ngdepends not only on the bisection bandwidth of the interconnection network, see\npage 38, but also on the communication protocol used and on the implementation of\nthe communication library

relatedness=0.785


'2.7 Caches and Memory Hierarchy 97\nWeak ordering models additionally relax the R→RandR→Worderings.\nThus, no completion order of the memory operations is guaranteed. To support pro-\ngramming, these models provide additional synchronization operations to ensure thefollowing properties:\n•All read and write operations which lie in the program before the synchronization\noperation are completed before the synchronization operation.\n•The synchronization operation is completed before read or write operations are\nstarted which lie in the program after the synchronization operation.\nThe advent of multicore processors has lead to an increased availability of parallel\nsystems and most processors provide hardware support for a memory consistency\nmodel. Often, relaxed consistency models are supported, as it is the case for the Pow-erPC architecture of IBM or the different Intel architectures. But different hardware\nmanufacturers favor different models, and there is no standardization as 

relatedness=0.774


'97. Heywood, T. and Ranka, S. A Practical Hierarchical Model of Parallel Computation.\nJournal of Parallel and Distributed Computing , 16:212–249, 1992.\n98. J.M.D. Hill, B. McColl, D.C. Stefanescu, M.W. Goudreau, K. Lang, S.B. Rao, T. Suel,\nT. Tsantilas, and R. Bisseling. BSPlib The BSB Programming Library. Technical Report\nTR-29-97, Oxford University, May 1997.\n99. M. Hill, W. McColl, D. Skillicorn, Questions and Answers about BSP. Scientiﬁc\nProgramming , 6(3):249–274, 1997.\n100. C.A.R. Hoare, Monitors: An Operating Systems Structuring Concept. Commun. ACM ,\n17(10):549–557, 1974.\n101. R. Hockney, A Fast Direct Solution of Poisson’s Equation Using Fourier Analysis. Journal\nof the ACM , 12:95–113, 1965.\n102. R.W. Hockney. The Science of Computer Benchmarking . SIAM, 1996.\n103. R. Hoffmann and T. Rauber. Fine-grained task scheduling using adaptive data structures. In\nProc. of Euro-Par , volume 5168 of Lecture Notes in Computer Science , pages 253–262.\nSpringer, 2008.\n104. 